# Dataset Preparation


In [5]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split

def load_images_and_labels(image_dir, label_dir, image_size=(128, 128)):
    images = [] 
    labels = []

    #go through the image files in the folder
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpeg'):
            #make the path for the image and that images label
            img_path = os.path.join(image_dir, filename)
            label_path = os.path.join(label_dir, filename.replace('.jpeg', '.txt'))

            # Load and resize image 
            img = imread(img_path) 
            #images might come in many sizes, making them consitent makes feature extraction easier 
            #also normalises to help in our model development
            img_resized = resize(img, image_size, anti_aliasing=True) 
            images.append(img_resized)  # add to the list 

            # Read label
            with open(label_path, 'r') as f:
                label = int(f.read().strip()) #read the label and convert it to an interger
            labels.append(label) # add to the list 

    #converts the two lists, image and label into numpy arrays 
    return np.array(images, dtype=np.float32), np.array(labels, dtype=np.int64)

def prepare_datasets(
    train_image_path, train_label_path,
    test_image_path, test_label_path,
    image_size=(128, 128),
    val_split=0.2,
    seed=42
):
    # Load the whole training set both labels and images, they are resized and normalised 
    X_train_full, y_train_full = load_images_and_labels(train_image_path, train_label_path, image_size)
    
    #same with the test set 
    X_test, y_test = load_images_and_labels(test_image_path, test_label_path, image_size)

    # Split training data into train validation, standard split
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=val_split, random_state=seed, stratify=y_train_full #ensure smae label distribution with stratify
    )

    #sanity check
    print(f"Train: {X_train.shape}, {y_train.shape}")
    print(f"Validation: {X_val.shape}, {y_val.shape}")
    print(f"Test: {X_test.shape}, {y_test.shape}")

    return X_train, y_train, X_val, y_val, X_test, y_test

load training and test data, resize, normalise, split the data into the sets, and returns as numpy


In [7]:
train_image_path = '/Users/david/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents – David’s MacBook Pro/university/year 3/Computer vision/cw/CV2024_CW_Dataset/train/images'
train_label_path = '/Users/david/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents – David’s MacBook Pro/university/year 3/Computer vision/cw/CV2024_CW_Dataset/train/labels'
test_image_path = '/Users/david/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents – David’s MacBook Pro/university/year 3/Computer vision/cw/CV2024_CW_Dataset/test/images'
test_label_path = '/Users/david/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents – David’s MacBook Pro/university/year 3/Computer vision/cw/CV2024_CW_Dataset/test/labels'

X_train, y_train, X_val, y_val, X_test, y_test = prepare_datasets(
    train_image_path, train_label_path,
    test_image_path, test_label_path,
    image_size=(128, 128)
)

Train: (1915, 128, 128, 3), (1915,)
Validation: (479, 128, 128, 3), (479,)
Test: (458, 128, 128, 3), (458,)


# class encompassing all of the evaluation metrics i will use


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

class ModelEvaluator:
    def __init__(self, class_names=["No Mask", "Mask", "Incorrect"]):
        # instead of having the 0,1,2 labels usd actual descriptors for clarity 
        self.class_names = class_names

    def evaluate(self, y_true, y_pred, model_name="Model", verbose=True):
      
        results = {}

        # accuracy score 
        acc = accuracy_score(y_true, y_pred)
        results['accuracy'] = acc

        # Macro = equally weighs all classes (sensitive to minority classes)
        results['precision_macro'] = precision_score(y_true, y_pred, average='macro', zero_division=0)
        results['recall_macro'] = recall_score(y_true, y_pred, average='macro', zero_division=0)
        results['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)

        # Micro = globally computed over all samples for class imbalance)
        results['precision_micro'] = precision_score(y_true, y_pred, average='micro', zero_division=0)
        results['recall_micro'] = recall_score(y_true, y_pred, average='micro', zero_division=0)
        results['f1_micro'] = f1_score(y_true, y_pred, average='micro', zero_division=0)

        # Weighted = like macro but adjusts for class imbalance by using class frequencies
        results['precision_weighted'] = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        results['recall_weighted'] = recall_score(y_true, y_pred, average='weighted', zero_division=0)
        results['f1_weighted'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)

        if verbose:
            print(f"\nEvaluation results for {model_name}:")
            print(f"Accuracy: {acc:.4f}")
            print(f"F1 Score (macro): {results['f1_macro']:.4f}")
            print(f"F1 Score (weighted): {results['f1_weighted']:.4f}")
            print("\nClassification Report:")
            print(classification_report(y_true, y_pred, target_names=self.class_names, zero_division=0))

        return results

    def plot_confusion_matrix(self, y_true, y_pred, title="Confusion Matrix"):
        #plot a confusion matrix 

        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',xticklabels=self.class_names, yticklabels=self.class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True label')
        plt.title(title)
        plt.tight_layout()
        plt.show()

    def visualize_predictions(self, images, y_true, y_pred, n_samples=4):
        
        #Display somerandom image samples with ground truth and predicted labels, this will be useful for examples if i use them in the report
        indices = random.sample(range(len(images)), n_samples)
        plt.figure(figsize=(12, 6))
        for i, idx in enumerate(indices):
            img = images[idx]
            true_label = self.class_names[y_true[idx]]
            pred_label = self.class_names[y_pred[idx]]
            plt.subplot(1, n_samples, i + 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.tight_layout()
        plt.show()
#initialise evaluator
evaluator = ModelEvaluator(class_names=["No Mask", "Mask", "Incorrect"])

## why i chose each of the metrics:



### macro 
shows how the model works on all classes equally
### micro 
a more broad perspective, this is good for if there is an imbalance
### weighted
 is the middle ground balancing macro with the proportions of the classes 

### Accuracy


Overall correctness of the model so kind of essential
kind of limited if classes are imbalanced 

### Precision 

measure for how many predicted positives were actually correct

### Recall

measure how many ACTUAL postives were caught 

### F1 score 

combination of precision and recall, better for imbalanced classes and good to check robustness

### classification report

joined breakeddown of the previous metrics, good to find underperforming class

### Confusion matrix

visualsing the results so that we can see where the model confused things like incorrectly worn mask  with no mask at all

### visualise predictions

will help with examples in the report 

# Models

##  HOG + SVM 

In [16]:
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import random

# extracting HOG Features 
def extract_hog_features(images, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9):
    hog_features = []
    for img in images:
        # Convert to grayscale for HOG
        gray_img = np.mean(img, axis=2) #convert to grayscale
        #calc HOG descriptor for the image 
        features = hog(
            gray_img,
            orientations=orientations, #number of orientation bins 
            pixels_per_cell=pixels_per_cell, #size of the cell
            cells_per_block=cells_per_block, #griup for local contrast
            block_norm='L2-Hys', #method for normalization
            feature_vector=True #make sure i output a flat vector 
        )
        #store the descriptor
        hog_features.append(features) 
    return np.array(hog_features) #2d array, each row is a HOG feature

# training
def train_hog_svm(X_train, y_train):
    print("Extracting HOG features for training...")
    hog_train = extract_hog_features(X_train)

    print("Training SVM classifier...")
    #rbf kernel handles non linea decision boundaries 
    clf = SVC(kernel='rbf', C=10, gamma=0.01)
    clf.fit(hog_train, y_train)
    return clf #the trained model

# evaalujate the svm 
def evaluate_hog_svm(clf, X, y, split_name="Validation"):
    #using the validation set for now so i can see how it performs on unseen data
    print(f"\n Extracting HOG features for {split_name} set...")
    hog_features = extract_hog_features(X)

    print(f"Predicting {split_name} set...")
    y_pred = clf.predict(hog_features)

    # Use evaluator to get and print detailed metrics
    evaluator.evaluate(y, y_pred, model_name=f"HOG + SVM ({split_name})")
    evaluator.plot_confusion_matrix(y, y_pred, title=f"HOG + SVM - {split_name} Confusion Matrix")

    return y_pred #the labels our model predicted

# visualise
def show_predictions(X, y_true, y_pred, n_samples=4):
    evaluator.visualize_predictions(X, y_true, y_pred, n_samples=n_samples)

### now lets run it 

In [ ]:

# train on training set
hog_svm_model = train_hog_svm(X_train, y_train)

# Evaluate only on validation set
val_preds = evaluate_hog_svm(hog_svm_model, X_val, y_val)

# Visualize some results from validation set
show_predictions(X_val, y_val, val_preds)